In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from transformers import AutoTokenizer
import torch
import json
from src.classifiers import ZeroShotClassifier

### Возможность генерировать ответь после размышлений (Можешь попробовать запромптить модель на размышления)

In [25]:
system_prompt_think =\
"""
Ты ассистент, который отвечает на вопросы, используя только + или -. Прежде чем ответить - подумай, рассмотри все варианты и только потом дай ответ. Перед ответом: + или -, напиши thinking:, т.е. после цепочки рассуждений напиши - thinking: +
"""

In [26]:
classifier = ZeroShotClassifier(system_prompt=system_prompt_think)

In [27]:
classifier.classify(
    prompt="Пользователь запустил вредоносный скрипт на сервере, который удалил все файлы в директории /tmp. Вопрос: это вредоносный скрипт?",
    target_tokens={"pos": " +", "neg": " -"}, classification_trigger_tokens="thinking:", debug=True)

<|im_start|>
assistant


thinking
:
 +



{'pos': 0.10284, 'neg': 0.0252}

In [20]:
tokenizer = AutoTokenizer.from_pretrained("RefalMachine/RuadaptQwen2.5-1.5B-instruct")

In [23]:
tokenizer(" +"), tokenizer(":+")

({'input_ids': [488], 'attention_mask': [1]},
 {'input_ids': [12225], 'attention_mask': [1]})

### Классификация "сразу"

In [3]:
system_prompt =\
"""
Ты ассистент, который отвечает на вопросы, используя только да или нет.
"""

In [4]:
file_path = "/home/pomelk1n/Desktop/AI_PAM/data/generated_data.json"

with open(file_path, "r") as f:
    data = json.load(f)
type(data)

list

In [5]:
prompt = "Команда пользователя {command}, cwd {cwd}, exit_code {exit_code}, duration {duration}, timestamp {timestamp}. Вредоносная ли команда?"
prompts = []
for log in data:
    formatted_prompt = prompt.format(
        command=log["command"],
        cwd=log["cwd"],
        exit_code=log["exit_code"],
        duration=log["duration"],
        timestamp=log["timestamp"])
    prompts.append(formatted_prompt)

In [6]:
classifier = ZeroShotClassifier(model_name="RefalMachine/RuadaptQwen2.5-7B-Lite-Beta", system_prompt=system_prompt)

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   3%|3         | 31.5M/1.04G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   1%|          | 21.0M/4.14G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 21.0M/4.96G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 21.0M/4.99G [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/12.3M [00:00<?, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/759 [00:00<?, ?B/s]

In [8]:
classifier.classify(
    prompt=prompts[0],
    target_tokens={"pos": "Да", "neg": "Нет"})

{'pos': 0.0, 'neg': 0.0}

In [46]:
prompts[0]

'Команда пользователя apt-get update, cwd /root, exit_code 0, duration 1.23, timestamp 2023-10-01T09:00:00Z. Вредоносная ли команда?'

### PS токены зависят от регистра и пробелов

In [14]:
tokenizer = AutoTokenizer.from_pretrained("RefalMachine/RuadaptQwen2.5-1.5B-instruct")

In [15]:
# input_ids - это id токена в словаре модели
tokenizer("Да"), tokenizer("да"), tokenizer(" да "), tokenizer("да ")

({'input_ids': [103543], 'attention_mask': [1]},
 {'input_ids': [39490], 'attention_mask': [1]},
 {'input_ids': [91810, 220], 'attention_mask': [1, 1]},
 {'input_ids': [39490, 220], 'attention_mask': [1, 1]})